In [1]:
%pip install langchain faiss-cpu transformers torch accelerate sentence-transformers dashscope

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install dashscope langchain-community

Looking in indexes: https://mirrors.tencent.com/pypi/simple/

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# -*- coding: utf-8 -*-
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms import Tongyi  # 新版接口
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# ================== 1. 加载并切分文档 ==================
def load_and_split_documents(file_path="laws.txt"):
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# ================== 2. 初始化 Embedding 模型 ==================
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

# ================== 3. 构建向量数据库 ==================
def build_vectorstore(docs, embeddings):
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

# ================== 4. 初始化 Qwen LLM（Tongyi）==================
def get_llm(api_key):
    llm = Tongyi(model_name="qwen-max", dashscope_api_key=api_key)
    return llm

# ================== 5. 定义 Prompt 模板 ==================
def get_prompt_template():
    template = """用户问题：{question}

相关法律条文：{context}

请简要判断该政策是否违反公平竞争原则，并说明依据：
"""
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    return prompt

# ================== 6. 构建 QA 链 ==================
def build_qa_chain(vectorstore, llm, prompt):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# ================== 主函数入口 ==================
def main():
    # 设置 DashScope API Key
    DASHSCOPE_API_KEY = "sk-7fb2aee47f5d4531855a7ac3412249fe"  # 替换为你的实际 API Key
    os.environ["DASHSCOPE_API_KEY"] = DASHSCOPE_API_KEY

    # 步骤 1：加载并切分文档
    print("正在加载并切分文档...")
    docs = load_and_split_documents()

    # 步骤 2：初始化 Embedding 模型
    print("正在加载 Embedding 模型...")
    embeddings = get_embeddings()

    # 步骤 3：构建 FAISS 向量数据库
    print("正在构建向量数据库...")
    vectorstore = build_vectorstore(docs, embeddings)

    # 步骤 4：初始化 LLM
    print("正在加载 Qwen 模型...")
    llm = get_llm(DASHSCOPE_API_KEY)

    # 步骤 5：定义 Prompt 模板
    print("正在构建 Prompt 模板...")
    prompt = get_prompt_template()

    # 步骤 6：构建 QA 链
    print("正在整合 RetrievalQA 链...")
    qa_chain = build_qa_chain(vectorstore, llm, prompt)

    # 示例查询
    query = "某地出台政策，要求外地企业必须在本地设立分支机构才能参与政府采购，是否合规？"

    print("\n开始推理...\n")
    result = qa_chain.run(query)

    print("回答结果：")
    print(result)

if __name__ == "__main__":
    main()

正在加载并切分文档...
正在加载 Embedding 模型...


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


正在构建向量数据库...
正在加载 Qwen 模型...
正在构建 Prompt 模板...
正在整合 RetrievalQA 链...

开始推理...



/tmp/ipykernel_6989/1779634094.py:90: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)


回答结果：
**判断：该政策违反公平竞争原则。**

**依据：**

1. **违反限制商品、要素自由流动的原则（第二条）：**
   - 根据相关法律条文第二条第（二）项，政策不得“排斥、限制、强制或者变相强制外地经营者在本地投资经营或者设立分支机构”。要求外地企业必须在本地设立分支机构才能参与政府采购，属于变相强制外地经营者设立分支机构的行为，限制了外地企业平等参与市场竞争的权利。
   - 同时，根据第二条第（三）项，政策不得“排斥、限制或者变相限制外地经营者参加本地政府采购、招标投标”。该政策通过设置分支机构的前置条件，实际上对外地企业参与政府采购构成了歧视性限制。

2. **违反合理准入条件的原则（第一条）：**
   - 根据第一条第（四）项，政策措施不得“设置不合理或者歧视性的准入、退出条件”。要求外地企业在本地设立分支机构作为参与政府采购的前提条件，显然属于不合理和歧视性的准入条件。

3. **不符合例外情形（第五条）：**
   - 该政策并未涉及维护国家安全、促进科技进步、实现社会公共利益等法定例外情形，也未提供对公平竞争影响更小的替代方案，因此不能依据第五条的规定作为例外处理。

**结论：**
该政策违反了公平竞争审查的相关规定，侵害了外地企业的合法权益，应予以调整或废止，以确保市场环境的公平与开放。
